## Project setup
Configure imports and setup the connection to the Alpaca API

In [1]:
import os
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame
from alpaca_trade_api.stream import Stream
import backtrader as bt
import matplotlib
import pandas as pd
from alpaca_trade_api import TimeFrameUnit

In [2]:
load_dotenv()

alpaca_endpoint = 'https://paper-api.alpaca.markets'
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

rest_api = REST(alpaca_api_key, alpaca_secret_key, alpaca_endpoint)

# Configure Base algorithim

In [ ]:
# Creates a Function to perform backtesting using the alpaca API
def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
	"""params:
          strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
          symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
          start: start date of backtest in format 'YYYY-MM-DD'
          end: end date of backtest in format: 'YYYY-MM-DD'
          timeframe: the timeframe the strategy trades on (size of bars) -
                      1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
          cash: the starting cash of backtest
    """

	# initialize backtrader broker
	cerebro = bt.Cerebro(stdstats=True)
	cerebro.broker.setcash(cash)

	# add strategy
	cerebro.addstrategy(strategy)

	# add analytics
	# cerebro.addobserver(bt.observers.Value)
	# cerebro.addobserver(bt.observers.BuySell)
	cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
	# historical data request
	if type(symbols) == str:
		symbol = symbols
		alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
		data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
		cerebro.adddata(data)
	elif type(symbols) == list or type(symbols) == set:
		for symbol in symbols:
			alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
			data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
			cerebro.adddata(data)


	# run
	initial_portfolio_value = cerebro.broker.getvalue()
	print(f'Starting Portfolio Value: {initial_portfolio_value}')
	results = cerebro.run()
	final_portfolio_value = cerebro.broker.getvalue()
	print(f'Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%')

	strat = results[0]
	print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis()['sharperatio'])
	cerebro.plot(iplot= False)

In [7]:
spy_bars = rest_api.get_bars('SPY', TimeFrame(5, TimeFrameUnit.Minute), '2022-02-01', '2022-02-24').df
spy_bars

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-02-01 09:00:00+00:00,450.200,450.55,450.09,450.220,9050,78,450.407316
2022-02-01 09:05:00+00:00,450.260,450.26,449.90,449.950,28362,111,449.981411
2022-02-01 09:10:00+00:00,449.900,450.23,449.88,450.050,4659,41,450.037246
2022-02-01 09:15:00+00:00,449.880,450.08,449.83,449.870,1404,38,449.915406
2022-02-01 09:20:00+00:00,449.750,449.75,449.25,449.250,1947,30,449.357011
...,...,...,...,...,...,...,...
2022-02-24 20:05:00+00:00,424.540,424.75,424.19,424.230,1947482,22604,424.169355
2022-02-24 20:10:00+00:00,424.230,424.79,424.07,424.720,1783091,19490,424.128639
2022-02-24 20:15:00+00:00,424.725,424.81,424.11,424.755,2204455,19744,423.878885


In [ ]:
def VolBreakout(today_open, prev_high, prev_low,  multiplier):

    top_range_calc = prev_high * multiplier
    bottom_range_calc = prev_low * multiplier

    top_of_range = today_open + top_range_calc
    bottom_of_range = today_open - bottom_range_calc

    if  > top_of_range:

